# Classifier with automatically Drift Detection Implementation

In [ ]:
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment
from sklearn.datasets import make_classification
import tempfile
import pandas as pd
import mlflow 

In [ ]:
# create experiment
experiment_name = "model_with_drift_detection"
experiment = get_or_create_experiment(
    experiment_name = experiment_name,
    tags = {
        "proejct_name": "automatic_model_monitoring",
        "topic":"pyfunc_flavor",
        "mlflow.note.content": "This experiment is used to track the model with drift detection"
    }
)

### Create demo dataset

In [ ]:
x , y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=10, n_clusters_per_class=1)

feature_names = [f"feature_{i}" for i in range(x.shape[1])]

df = pd.DataFrame(x, columns=feature_names)
df["target"] = y

### Create custom model

In [ ]:
from mlflow_for_ml_dev.utils.utils import get_root_project
from mlflow_for_ml_dev.experiments.custom_models import CustomClassifier


# start run
with mlflow.start_run(run_name="custom_classifier", experiment_id=experiment.experiment_id) as run:
    # train model
    custom_clf = CustomClassifier(run_id = run.info.run_id)
    custom_clf.fit_estimator(df[feature_names], df["target"])

    # save dataset
    df.to_csv("training_data.csv", index=False)

    mlflow.pyfunc.log_model(python_model=custom_clf,
                            artifact_path=custom_clf.__class__.__name__,
                            code_path=[(get_root_project() / "mlflow_for_ml_dev" / "experiments" /"custom_models.py" ).as_posix()],
                            artifacts={"x_ref_path":"training_data.csv"})

In [ ]:
loaded_model = mlflow.pyfunc.load_model("runs:/" + run.info.run_id + "/" + "CustomClassifier")

In [ ]:
loaded_model.predict(df[feature_names])

: 